In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xarray as xr
import numpy as np
from tqdm import tqdm

In [20]:
train_ds = test_ds_v4 = xr.open_dataset('/projekt_agmwend/home_rad/Joshua/test_ds_v5.nc', chunks={'time' : -1, 'x' : 250, 'y' : 250}, engine='h5netcdf')


In [21]:
from helper import watershed_transformation
from tqdm import tqdm

def segment_training_data(ds):
    
    segments_array = np.zeros(ds['BT_2D'].isel(band=0).shape)   

    #give the segments a unique id 


    for i, time in tqdm(enumerate(ds.time)):
        segments = watershed_transformation(ds['BT_2D'].isel(time=i, band=0).values, feature_separation=5)
        segments_array[i, :, :] = segments + segments_array.max()


    ds['segmentation'] = (('time', 'x', 'y'), segments_array)
    
    return ds

test_ds_v4 = segment_training_data(test_ds_v4)

#test_ds_v4 = test_ds_v4.stack(z=('x', 'y', 'time')).isel(band=0).dropna('z' , how='any')  



63it [01:13,  1.17s/it]


In [22]:
train_ds_flat = train_ds.stack(feature=['x', 'y', 'time'])

In [8]:
train_ds_flat

<xarray.Dataset>
Dimensions:       (band: 5, feature: 20282535)
Coordinates:
  * feature       (feature) MultiIndex
  - x             (feature) int64 0 0 0 0 0 0 0 ... 634 634 634 634 634 634 634
  - y             (feature) int64 0 0 0 0 0 0 0 ... 506 506 506 506 506 506 506
  - time          (feature) datetime64[ns] 2022-03-20T10:45:00 ... 2022-04-10...
Dimensions without coordinates: band
Data variables:
    BT_2D         (band, feature) float32 dask.array<chunksize=(5, 2651103), meta=np.ndarray>
    BT_Center     (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    alt           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lat           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lon           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    yaw           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    label         (feature) uint8 dask.array<chunksize=(2651103,), meta=np.ndarray>
    segmentation  (feature) float64 398.0 5.424e+03 ... 2.424e+05 2.465e+05
Attributes: (12/21)
    Conventions:             "CF-1.8"
    author:                  Michael Schäfer, Sophie Rosenburg, André Ehrlich...
    campaign:                HALO-AC3
    comment_1:               Band wavelength range: 7.70 - 12.00 micrometers
    comment_2:               Flight direction is towards positive y-direction...
    comment_3:               Thermal infrared imager information: focal lengt...
    ...                      ...
    research_flight_day:     20220316
    research_flight_number:  RF06
    source:                  Airborne imaging with the VELOX system
    title:                   Two-dimensional cloud-top or surface brightness ...
    variable:                BT_2D, BT_Center
    version:                 Version v3.0 from 2023-09-11

In [9]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=16,
    threads_per_worker=1,
    memory_limit='8GB'
)

client = Client(cluster)

/home/jomueller/.conda/envs/jomueller_env/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33487 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/projekt_agmwend/home_rad/Joshua/VELOX_Arctic_Surface_Classification/source/dask-worker-space/worker-0jdj33io', purging
distributed.diskutils - INFO - Found stale lock file and directory '/projekt_agmwend/home_rad/Joshua/VELOX_Arctic_Surface_Classification/source/dask-worker-space/worker-1y5gfmba', purging
distributed.diskutils - INFO - Found stale lock file and directory '/projekt_agmwend/home_rad/Joshua/VELOX_Arctic_Surface_Classification/source/dask-worker-space/worker-2y62vhpb', purging
distributed.diskutils - INFO - Found stale lock file and directory '/projekt_agmwend/home_rad/Joshua/VELOX_Arctic_Surface_Classification/source/dask-worker-space/worker-ht865gwt', purging
/hom

In [23]:
train_ds_flat = train_ds_flat.persist()

In [78]:
import xarray as xr
import dask
import dask.array as da
import numpy as np
from dask.diagnostics import ProgressBar

def calc_seg_stats(ds):
    segments = np.unique(ds.segmentation)

    # Define a function to compute statistics for each segment
    def compute_stats(segment):
        subset = ds.where(ds.segmentation == segment, drop=True)
        mean = subset.BT_2D.mean().values
        std = subset.BT_2D.std().values
        mini = subset.BT_2D.min().values
        maxi = subset.BT_2D.max().values
        size = subset.feature.size
        label_counts = np.unique(subset.label, return_counts=True)[1]
        label = np.argmax(label_counts)
        return mean, std, mini, maxi, size, label

    # Use Dask to parallelize the computation
    results = [dask.delayed(compute_stats)(segment) for segment in segments]

    # Compute the results in parallel
    with ProgressBar():
        stats = dask.compute(*results)

    # Combine the results into a dataset
    ds_stats = xr.Dataset(
        {
            'mean': (('segmentation'), stats[0]),
            'std': (('segmentation'), stats[1]),
            'min': (('segmentation'), stats[2]),
            'max': (('segmentation'), stats[3]),
            'size': (('segmentation'), stats[4]),
            'label': (('segmentation'), stats[5])
        },
        coords={'segmentation': segments}
    )

    return ds_stats

# Call the function to calculate segment statistics
with ProgressBar():
    ds_stats = calc_seg_stats(train_ds_flat)



AttributeError: module 'dask.array' has no attribute 'delayed'

In [69]:
def calc_seg_stats(ds):
    segments = np.unique(ds.segmentation)

    out = np.zeros((len(segments), 6))

    for i in tqdm(range(1, len(segments) + 1)):

        subset = ds.where(ds.segmentation == i, drop=True)
        out[i, 0] = subset.BT_2D.mean().values
        out[i, 1] = subset.BT_2D.std().values
        out[i, 2] = subset.BT_2D.min().values
        out[i, 3] = subset.BT_2D.max().values
        out[i, 4] = subset.feature.size
        out[i, 5] = np.argmax(np.unique(subset.label, return_counts=True)[1])   

    ds = xr.Dataset({'mean' : (('segmentation'), out[:, 0]),
                        'std' : (('segmentation'), out[:, 1]),
                        'min' : (('segmentation'), out[:, 2]),
                        'max' : (('segmentation'), out[:, 3]),
                        'size' : (('segmentation'), out[:, 5]),
                        'label' : (('segmentation'), out[:, 6])})
    return ds

    
ds =  calc_seg_stats(train_ds_flat)



  0%|          | 16/250005 [00:09<40:13:32,  1.73it/s]


KeyboardInterrupt: 

In [68]:
train_ds_flat

<xarray.Dataset>
Dimensions:       (band: 5, feature: 20282535)
Coordinates:
  * feature       (feature) MultiIndex
  - x             (feature) int64 0 0 0 0 0 0 0 ... 634 634 634 634 634 634 634
  - y             (feature) int64 0 0 0 0 0 0 0 ... 506 506 506 506 506 506 506
  - time          (feature) datetime64[ns] 2022-03-20T10:45:00 ... 2022-04-10...
Dimensions without coordinates: band
Data variables:
    BT_2D         (band, feature) float32 dask.array<chunksize=(5, 2651103), meta=np.ndarray>
    BT_Center     (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    alt           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lat           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lon           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    yaw           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    label         (feature) uint8 dask.array<chunksize=(2651103,), meta=np.ndarray>
    segmentation  (feature) float64 398.0 5.424e+03 ... 2.424e+05 2.465e+05
Attributes: (12/21)
    Conventions:             "CF-1.8"
    author:                  Michael Schäfer, Sophie Rosenburg, André Ehrlich...
    campaign:                HALO-AC3
    comment_1:               Band wavelength range: 7.70 - 12.00 micrometers
    comment_2:               Flight direction is towards positive y-direction...
    comment_3:               Thermal infrared imager information: focal lengt...
    ...                      ...
    research_flight_day:     20220316
    research_flight_number:  RF06
    source:                  Airborne imaging with the VELOX system
    title:                   Two-dimensional cloud-top or surface brightness ...
    variable:                BT_2D, BT_Center
    version:                 Version v3.0 from 2023-09-11

In [75]:
import numpy as np
import xarray as xr
from tqdm import tqdm

def calc_seg_stats(ds):
    print(ds)   
    print(ds.shape)
    segments = np.unique(ds['segmentation'])
    out = np.zeros((len(segments), 6))

    for i in tqdm(range(len(segments))):
        subset = ds.where(ds.segmentation == segments[i], drop=True)
        out[i-1, 0] = subset.BT_2D.mean().values
        out[i-1, 1] = subset.BT_2D.std().values
        out[i-1, 2] = subset.BT_2D.min().values
        out[i-1, 3] = subset.BT_2D.max().values
        out[i-1, 4] = subset.feature.size
        out[i-1, 5] = np.argmax(np.unique(subset.label, return_counts=True)[1])

    ds_out = xr.Dataset({'mean': (('segmentation',), out[:, 0]),
                         'std': (('segmentation',), out[:, 1]),
                         'min': (('segmentation',), out[:, 2]),
                         'max': (('segmentation',), out[:, 3]),
                         'size': (('segmentation',), out[:, 4]),
                         'label': (('segmentation',), out[:, 5])},
                        coords={'segmentation': segments})
    return ds_out

num_segments = len(np.unique(train_ds_flat.segmentation))

# Parallelize the function using apply_ufunc
ds = xr.apply_ufunc(
    calc_seg_stats,
    train_ds_flat,
    input_core_dims=[['feature']],
    output_core_dims=[['segmentation']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    dask_gufunc_kwargs={'output_sizes': {'segmentation': num_segments}, 
                        'allow_rechunk': True}  
)

[   398.   5424.   8524. ... 237762. 242396. 246492.]
(20282535,)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [74]:
train_ds_flat

<xarray.Dataset>
Dimensions:       (band: 5, feature: 20282535)
Coordinates:
  * feature       (feature) MultiIndex
  - x             (feature) int64 0 0 0 0 0 0 0 ... 634 634 634 634 634 634 634
  - y             (feature) int64 0 0 0 0 0 0 0 ... 506 506 506 506 506 506 506
  - time          (feature) datetime64[ns] 2022-03-20T10:45:00 ... 2022-04-10...
Dimensions without coordinates: band
Data variables:
    BT_2D         (band, feature) float32 dask.array<chunksize=(5, 2651103), meta=np.ndarray>
    BT_Center     (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    alt           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lat           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    lon           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    yaw           (band, feature) float32 dask.array<chunksize=(5, 20282535), meta=np.ndarray>
    label         (feature) uint8 dask.array<chunksize=(2651103,), meta=np.ndarray>
    segmentation  (feature) float64 398.0 5.424e+03 ... 2.424e+05 2.465e+05
Attributes: (12/21)
    Conventions:             "CF-1.8"
    author:                  Michael Schäfer, Sophie Rosenburg, André Ehrlich...
    campaign:                HALO-AC3
    comment_1:               Band wavelength range: 7.70 - 12.00 micrometers
    comment_2:               Flight direction is towards positive y-direction...
    comment_3:               Thermal infrared imager information: focal lengt...
    ...                      ...
    research_flight_day:     20220316
    research_flight_number:  RF06
    source:                  Airborne imaging with the VELOX system
    title:                   Two-dimensional cloud-top or surface brightness ...
    variable:                BT_2D, BT_Center
    version:                 Version v3.0 from 2023-09-11